# 要約 
このJupyter Notebookは、Kaggleの「LMSYS - Chatbot Arena」コンペティションにおけるXGBoostを使用したベースラインモデルの構築に関連しています。具体的には、ユーザーが好む応答を予測するためのモデルをデータを用いてトレーニングし、その精度を評価することを目的としています。

### 取り組んでいる問題
Notebookでは、異なる大規模言語モデル（LLM）が生成した応答の中から、ユーザーがどの応答を好むかを予測するという課題に対処しています。これは、データセットに含まれる「prompt」、「response_a」、「response_b」に基づいて、どちらの応答が優れているかを識別するためのものであり、最終的にはその選好をモデル学習することに繋がります。

### 使用される手法とライブラリ
- **ライブラリ**: 
  - `numpy`, `pandas`: データ操作のため。
  - `nltk`: 自然言語処理とテキストトークン化。
  - `matplotlib`, `seaborn`: データビジュアライゼーション。
  - `xgboost`: 機械学習モデルの構築用。
  - `sklearn`: 特徴量の抽出やモデル評価に利用。

- **モデルの構築**:
  - データを読み込み、前処理を行った後、特徴量エンジニアリングを実施。コサイン類似度やJaccard類似度を計算し、n-gramの重複数などの特徴を生成します。
  - `XGBoost`を使用して多クラスの分類モデルを構築し、ストラティファイドKフォールド交差検証を用いてモデルの性能を評価します。
  - 評価基準として対数損失（log loss）を使用し、モデルの精度を測定している点が特徴的です。

### 結果の提出
最終的には、テストデータに基づいた予測結果を生成し、指定されたフォーマットで「submission.csv」として保存します。このプロセスにより、モデルがどの応答を「勝者」として予測するかを示すことができます。

このNotebookは、異なる言語モデル間でのユーザーの選好を理解し、予測できるモデルを構築するための体系的なアプローチを採用しています。

---


# 用語概説 
以下に、jupyter notebookの中で使われている専門用語やコンセプトについて、特に初心者がつまずきやすいと思われる項目の簡単な解説を示します。

1. **n-gram**: 連続するn個の単語の組み合わせを指します。例えば、"I love AI"という文から生成される1-gram（単語単位）は "I", "love", "AI"、2-gram（隣接する2単語の組み合わせ）は "I love", "love AI" となります。n-gramはテキスト解析に用いられ、頻出のフレーズを把握するのに役立ちます。

2. **TF-IDF (Term Frequency-Inverse Document Frequency)**: 単語の重要度を計算するための指標です。特定の文書内での単語の出現頻度（TF）と、その単語が他の文書に出現する頻度の逆数（IDF）を掛け合わせることで、頻繁に使われるが特有の単語を重視します。情報検索やテキストマイニングでよく使われます。

3. **cosine similarity**: 2つのベクトル間のコサイン角度を基にした類似性を測る指標です。コサインの値が1に近いほど、2つのベクトルは似ていると評価されます。文書間の類似性を測定する際によく使用されます。

4. **Jaccard similarity**: 2つの集合の共通部分のサイズを、両者の和集合のサイズで割ったものです。たとえば、文書内の単語の集合を比較する際に利用され、重複する単語の割合を計算します。

5. **StratifiedKFold**: StratifiedKFoldは、データをK個の部分に分けるときに、各部分におけるクラスの比率を保つ方法です。これにより、訓練データと検証データの分布が似るため、モデルの評価が安定します。

6. **early stopping**: モデルの訓練過程で、特定の評価指標（例: 検証データのロス）が一定の回数改善しなかった場合に、訓練を中止する技術です。オーバーフィッティングを防ぐために役立ちます。

7. **XGBoost**: "Extreme Gradient Boosting"の略で、決定木をベースにした強力な機械学習アルゴリズムです。加重されたボースト法に基づいており、高速で高精度な予測を実現するため、多くのデータサイエンスコンペティションで使用されます。

8. **log loss**: またはクロスエントロピー損失関数、モデルの確率的予測がどれだけ正解から離れているかを評価する指標です。特に二値分類や多クラス分類で使われ、値が小さいほどモデルの予測が良好であることを示します。

これらの用語は、特に実務経験がない初心者にとっては少し馴染みが薄いものが多いですが、機械学習や深層学習のプロセスの中で非常に重要な概念です。理解しておくことで、今後の学習や実装がスムーズになるでしょう。

---


# LMSYS | XGB ベースライン

# 1. ライブラリ



In [ ]:
import gc
import os
import re
import numpy as np
import pandas as pd

import nltk
from nltk.util import ngrams
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

# 2. 設定



In [ ]:
class config:
    root = "/kaggle/input/lmsys-chatbot-arena/"
    train_path = os.path.join(root, "train.csv")  # 訓練データのファイルパス
    test_path = os.path.join(root, "test.csv")    # テストデータのファイルパス
    sample_submission_path = os.path.join(root, "sample_submission.csv")  # サンプル提出ファイルのパス
    seed = 42  # 乱数シード
    n_splits = 10  # クロスバリデーションの分割数

# 3. データの読み込み



In [ ]:
train = pd.read_csv(config.train_path)  # 訓練データを読み込む
test = pd.read_csv(config.test_path)    # テストデータを読み込む
sample_submission = pd.read_csv(config.sample_submission_path)  # サンプル提出データを読み込む

if test.shape[0] < 10:  # テストデータの行数が10未満なら
    train = train.iloc[:10000]  # 訓練データの最初の10000行を使用する
    
def process(input_str):
    stripped_str = input_str.strip('[]')  # 角括弧を削除
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # 文字列を分割し、前後のダブルクォートを削除
    return  ' '.join(sentences)  # スペースで結合して返す

# "prompt", "response_a", "response_b"のデータを加工
train["prompt"] = train["prompt"].apply(process)
train["response_a"] = train["response_a"].apply(process)
train["response_b"] = train["response_b"].apply(process)

test["prompt"] = test["prompt"].apply(process)
test["response_a"] = test["response_a"].apply(process)
test["response_b"] = test["response_b"].apply(process)

print(f"train shape: {train.shape}")  # 訓練データの形状を表示
print(f"test shape: {test.shape}")    # テストデータの形状を表示
print("-" * 90)  # 区切り線を表示
print(f"train missing values: {train.isnull().sum().sum()}")  # 訓練データの欠損値の合計を表示
print(f"test missing values: {test.isnull().sum().sum()}")    # テストデータの欠損値の合計を表示
print("-" * 90)

train.head()  # 訓練データの先頭5行を表示

# 4. 特徴量エンジニアリング



In [ ]:
class Preprocessor:

    def cosine_sim(self, text1: str, text2: str):
        try:
            vectorizer = TfidfVectorizer(ngram_range=(1, 3))  # TF-IDFベクトライザーを定義 (1-3のn-gram)
            vectorizer.fit([text1, text2])  # テキストを基にベクトライザーをフィッティング
            output = vectorizer.transform([text1, text2]).toarray()  # テキストをベクトル化
            cos_sim = cosine_similarity(output)  # コサイン類似度を計算
            return cos_sim[0][1]  # テキスト1とテキスト2のコサイン類似度を返す
        except:
            return np.nan  # エラーが発生した場合はNaNを返す

    def jaccard_sim(self, text1: str, text2: str):
        set1 = set(text1.split())  # テキスト1を単語の集合に変換
        set2 = set(text2.split())  # テキスト2を単語の集合に変換
        intersection = set1.intersection(set2)  # 共通する単語を取得
        union = set1.union(set2)  # 単語の和集合を取得
        return len(intersection) / len(union)  # Jaccard類似度を計算して返す
    
    def count_new_lines(self, text: str) -> int:
        return text.count('\\n')  # テキスト内の改行の数をカウントして返す 

    def count_quotes(self, text: str) -> int:
        single_quote_pattern = r"'(.*?)'"  # 単一引用符のパターン
        double_quote_pattern = r'"(.*?)"'   # 二重引用符のパターン
        single_quotes = re.findall(single_quote_pattern, text)  # 単一引用符の全てを取得
        double_quotes = re.findall(double_quote_pattern, text)  # 二重引用符の全てを取得
        total_quotes = len(single_quotes) + len(double_quotes)  # 合計の引用符の数を計算
        return len(single_quotes) + len(double_quotes)  # 合計数を返す

    def tokenize(self, text: str):
        return nltk.word_tokenize(text.lower())  # テキストを小文字にしてトークン化 

    def generate_ngrams(self, text: str, n: int):
        tokens = self.tokenize(text)  # トークン化されたテキストを取得
        return list(ngrams(tokens, n))  # n-gramを生成して返す

    def count_ngram_overlaps(self, text1: str, text2: str, n: int) -> int:
        try:
            ngrams1 = self.generate_ngrams(text1, n)  # テキスト1のn-gramを生成
            ngrams2 = self.generate_ngrams(text2, n)  # テキスト2のn-gramを生成
            counter1 = Counter(ngrams1)  # テキスト1のn-gramのカウントを作成
            counter2 = Counter(ngrams2)  # テキスト2のn-gramのカウントを作成
            overlap = counter1 & counter2  # 重複するn-gramを取得
            overlap_count = sum(overlap.values())  # 重複の合計数を計算
            return overlap_count  # 重複数を返す
        except:
            return 0  # エラーが発生した場合は0を返す
        
    def run(self, data: pd.DataFrame) -> pd.DataFrame:
        # "response_a"と"response_b"のn-gram重複数を計算し、新しいカラムに追加
        data["respa_respb_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 1), axis=1)
        data["respa_respb_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 2), axis=1)
        data["respa_respb_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 3), axis=1)

        data["respa_prompt_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 1), axis=1)
        data["respa_prompt_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 2), axis=1)
        data["respa_prompt_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 3), axis=1)

        data["respb_prompt_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 1), axis=1)
        data["respb_prompt_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 2), axis=1)
        data["respb_prompt_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 3), axis=1)
        
        data["respa_len"] = data["response_a"].apply(lambda x: len(self.tokenize(x)))  # "response_a"のトークン数をカウント
        data["respb_len"] = data["response_b"].apply(lambda x: len(self.tokenize(x)))  # "response_b"のトークン数をカウント
        data["prompt_len"] = data["prompt"].apply(lambda x: len(self.tokenize(x)))  # "prompt"のトークン数をカウント
        
        data["respa_new_lines"] = data["response_a"].apply(lambda x: self.count_new_lines(x))  # "response_a"の改行数をカウント
        data["respb_new_lines"] = data["response_b"].apply(lambda x: self.count_new_lines(x))  # "response_b"の改行数をカウント
        data["prompt_new_lines"] = data["prompt"].apply(lambda x: self.count_new_lines(x))  # "prompt"の改行数をカウント
        
        # 各長さの比率や差分を計算し新しいカラムに格納
        data["respa_prompt_len_ratio"] = data["respa_len"] / data["prompt_len"]  # "response_a"と"prompt"の長さ比
        data["respb_prompt_len_ratio"] = data["respb_len"] / data["prompt_len"]  # "response_b"と"prompt"の長さ比
        data["respa_respb_len_ratio"] = data["respa_len"] / data["respb_len"]  # "response_a"と"response_b"の長さ比
        
        data["respa_respb_len_diff"] = data["respa_len"] - data["respb_len"]  # "response_a"と"response_b"の長さの差
        data["respa_prompt_len_diff"] = data["respa_len"] - data["prompt_len"]  # "response_a"と"prompt"の長さの差
        data["respb_prompt_len_diff"] = data["respb_len"] - data["prompt_len"]  # "response_b"と"prompt"の長さの差
        
        data["respa_prompt_overlap_unigram_len_ratio"] = data["respa_prompt_overlap_unigram"] / data["prompt_len"]  # unigramの比率
        data["respa_prompt_overlap_bigram_len_ratio"] = data["respa_prompt_overlap_bigram"] / data["prompt_len"]  # bigramの比率
        data["respa_prompt_overlap_trigram_len_ratio"] = data["respa_prompt_overlap_trigram"] / data["prompt_len"]  # trigramの比率

        data["respb_prompt_overlap_unigram_len_ratio"] = data["respb_prompt_overlap_unigram"] / data["prompt_len"]  # unigramの比率
        data["respb_prompt_overlap_bigram_len_ratio"] = data["respb_prompt_overlap_bigram"] / data["prompt_len"]  # bigramの比率
        data["respb_prompt_overlap_trigram_len_ratio"] = data["respb_prompt_overlap_trigram"] / data["prompt_len"]  # trigramの比率
        
        data["overlap_unigram_diff"] = data["respa_prompt_overlap_unigram"] - data["respb_prompt_overlap_unigram"]  # unigramの差
        data["overlap_bigram_diff"] = data["respa_prompt_overlap_bigram"] - data["respb_prompt_overlap_bigram"]  # bigramの差
        data["overlap_trigram_diff"] = data["respa_prompt_overlap_trigram"] - data["respb_prompt_overlap_trigram"]  # trigramの差
        
        data["overlap_unigram_ratio"] = data["respb_prompt_overlap_unigram"] / data["respa_prompt_overlap_unigram"]  # unigramの比率
        data["overlap_bigram_ratio"] = data["respb_prompt_overlap_bigram"] / data["respa_prompt_overlap_bigram"]  # bigramの比率
        data["overlap_trigram_ratio"] = data["respb_prompt_overlap_trigram"] / data["respa_prompt_overlap_trigram"]  # trigramの比率
        
        data["respa_quotes"] = data["response_a"].apply(lambda x: self.count_quotes(x))  # "response_a"の引用符の数
        data["respb_quotes"] = data["response_b"].apply(lambda x: self.count_quotes(x))  # "response_b"の引用符の数
        data["prompt_quotes"] = data["prompt"].apply(lambda x: self.count_quotes(x))  # "prompt"の引用符の数
        
        # コサイン類似度とJaccard類似度を計算
        data["respa_respb_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_a"], x["response_b"]), axis=1)
        data["respa_respb_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_a"], x["response_b"]), axis=1)
        
        data["respa_prompt_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_a"], x["prompt"]), axis=1)
        data["respa_prompt_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_a"], x["prompt"]), axis=1)
        
        data["respb_prompt_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_b"], x["prompt"]), axis=1)
        data["respb_prompt_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_b"], x["prompt"]), axis=1)
        
        data["jaccard_sim_diff"] = data["respa_prompt_jaccard_sim"] - data["respb_prompt_jaccard_sim"]  # Jaccard類似度の差
        data["jaccard_sim_ratio"] = data["respb_prompt_jaccard_sim"] / data["respa_prompt_jaccard_sim"]  # Jaccard類似度の比率
        
        return data

In [ ]:
%%time
preprocessor = Preprocessor()  # Preprocessorのインスタンスを作成
train = preprocessor.run(train)  # 訓練データに前処理を適用
test = preprocessor.run(test)  # テストデータに前処理を適用
train.head()  # 訓練データの先頭5行を表示

In [ ]:
drop_cols = ["id", "response_a", "response_b", "prompt"]  # 削除するカラムのリスト
target_cols = ["winner_model_a", "winner_model_b", "winner_tie"]  # ターゲットカラムのリスト
target = "target"  # ターゲット名

train[target] = np.nan  # ターゲットカラムを初期化
for idx, t in enumerate(target_cols):  # ターゲットカラムを走査
    train.loc[train[t] == 1, target] = idx  # 勝者モデルをターゲットカラムに設定
train[target] = train[target].astype("int32")  # ターゲットカラムのデータ型をint32に変換
    
train.head()  # 訓練データの先頭5行を表示

# 5. モデリング



In [ ]:
X = train.drop(columns=target_cols + drop_cols + [target] + ["model_a", "model_b"], axis=1)  # 特徴量マトリックスを定義
y = train[target]  # ターゲットを定義
X_test = test.drop(columns=drop_cols, axis=1)  # テストデータから不要なカラムを削除

X = X.replace([-np.inf, np.inf], np.nan)  # 無限大をNaNに置き換え
X_test = X_test.replace([-np.inf, np.inf], np.nan)  # 無限大をNaNに置き換え

In [ ]:
cv = StratifiedKFold(n_splits=config.n_splits, shuffle=True, random_state=config.seed)  # ストラティファイドKフォールド交差検証の設定
test_preds = np.zeros(shape=(X_test.shape[0], y.nunique()))  # テストデータの予測結果を格納する配列
cv_scores = list()  # クロスバリデーションのスコアを格納するリスト

features = X.columns.tolist()  # 特徴量のリストを作成
feat_imp_df = pd.DataFrame({"feature": features})  # 特徴量の重要度を格納するデータフレームを作成

for idx, (train_idx, val_idx) in enumerate(cv.split(X, y)):  # 各Foldについて
    print(f"| Fold {idx+1} |".center(90, "="))  # 現在のFold番号を表示
    X_train, y_train = X.loc[train_idx], y.loc[train_idx]  # 訓練データを分割
    X_val, y_val = X.loc[val_idx], y.loc[val_idx]  # 検証データを分割

    print(f'train: {X_train.shape}')  # 訓練データの形状を表示
    print(f'val: {X_val.shape}')  # 検証データの形状を表示
    
    model = xgb.XGBClassifier(  # XGBoostの分類器モデルを定義
        objective='multi:softprob',  # 多クラス分類のための設定
        num_class=3,  # クラスの数
        eval_metric='mlogloss',  # 評価指標に対数損失を使用
        subsample=0.8,  # サンプリング比率
        n_estimators=650,  # 弱学習器の数
        learning_rate=0.045,  # 学習率
        max_depth=5,  # 木の深さ
        random_state=config.seed  # 乱数シード
    )
    
    model.fit(  # モデルを訓練
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],  # 訓練と検証データセット
        early_stopping_rounds=75,  # 早期停止の設定
        verbose=75  # 訓練過程を75ステップごとに表示
    )
    
    val_preds = model.predict_proba(X_val)  # 検証データに対する予測確率を計算
    val_log_loss = log_loss(y_val, val_preds, eps="auto")  # 検証データの対数損失を計算
    print(f"val log loss: {val_log_loss:.5f}")  # 検証データの対数損失を表示
    cv_scores.append(val_log_loss)  # クロスバリデーションのスコアを追加
    
    test_preds += model.predict_proba(X_test) / cv.get_n_splits()  # テストデータの予測を加算（平均化）

    feat_imp_df = feat_imp_df.merge(  # 特徴量の重要度を追加
        pd.DataFrame(
            {
                "feature": features,
                f"fold_{idx+1}_feat_imp": model.feature_importances_,  # 各Foldの特徴量重要度を取得
            }
        ),
        on=["feature"],
        how="left",
    )

print("="*90)  # 区切り線を表示
print(f"CV: {np.mean(cv_scores):.5f}")  # クロスバリデーションスコアの平均を表示

feat_imp_df["avg_importance"] = feat_imp_df.iloc[:, 1:].mean(axis=1)  # 各特徴の平均重要度を計算
plt.figure(figsize=(12, 10))  # グラフのサイズを設定
sns.barplot(
    data=feat_imp_df.sort_values(by="avg_importance", ascending=False).iloc[:50],  # 平均重要度でソートし上位50件を選択
    x="avg_importance",
    y="feature",
    color="royalblue",
    width=0.75,
)
plt.title("全Foldの特徴量の平均重要度", size=12)  # グラフのタイトル
plt.show()  # グラフを表示

# 6. 提出の保存



In [ ]:
for idx, t in enumerate(target_cols):  # 各ターゲットカラムについて
    sample_submission[t] = test_preds[:, idx]  # テストデータの予測結果をサンプル提出データに格納
sample_submission.head()  # サンプル提出データの先頭5行を表示

In [ ]:
sample_submission.to_csv("submission.csv", index=False)  # 提出ファイルをCSV形式で保存（インデックスなし）

---

# コメント

> ## Ilya Turaev
>
> こんにちは [@sercanyesiloz](https://www.kaggle.com/sercanyesiloz) ! 一つのXGBで素晴らしい結果です。コサイン類似度の関数がうまく動作していないようです。
>
> ```
> vectors = vectorizer.toarray()
> ```
>
> これを修正すれば、スコアが改善すると思いますか？それとも、他の特徴量だけで十分ですか？
>
> > ## Sercan Yeşilöz トピック作成者
> > 
> > こんにちは！そのコードでは、vectorizerは実際には訓練セットとテストセットでフィットした出力です。間違いはないと思います。
> >
> > > ## Ilya Turaev
> > > 
> > > 間違いなくそうすべきです、もしあなたがそれを次のように上書きする場合：
> > > 
> > > ```
> > > vectorizer = vectorizer.fit_transform([text1, text2])
> > > ```
> > > 
> > > そうでなければ、vectorizerはまだTfIdfオブジェクトであり、toarray()メソッドを適用しています。私は間違っていますか？
> > > 
> > > 
> > > ## Sercan Yeşilöz トピック作成者
> > > 
> > > 私は、その行の後でvectorizerがtfidfオブジェクトになるとは思いません、なぜならfit_transform関数を呼び出すと、変換された値が返されるからです。
> > > 
> > > 

---